## Step 13 가변 길이 인수(역전파 편)

In [10]:
import numpy as np

In [11]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))
            
        self.data = data
        self.grad = None 
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data) # y.grad = np.array(1.0)생략 가능

        funcs = [self.creator]
        while funcs:
            f = funcs.pop() # 함수를 가져온다.
            gys = [output.grad for output in f.outputs]#출력변수 outputs에 담겨있는 미분값들을 리스트에 담음
            gxs = f.backward(*gys) #함수 f의 역전파 호출
            if not isinstance(gxs, tuple): #튜플이 아니라면 튜플로 변환
                gxs = (gxs,)
            
            for x, gx in zip(f.inputs, gxs): #역전파로 전파되는 미분값을 Variable의 인스턴스 변수 grad에 저장해둠
                x.grad = gx

                if x.creator is not None:
                    funcs.append(x.creator)

In [12]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [13]:
class Function:
    def __call__(self, *inputs): #별표를 붙인다, 임의의 개수를 인수를 건네 함수 호출 가능
        xs = [x.data for x in inputs] #인수와 반환값 리스트로 변경
        ys = self.forward(*xs) #별표를 붙여 언팩, 리스트의 원소를 낱개로 풀어서 전달
        if not isinstance(ys, tuple): #튜플이 아닌 경우 추가 지원
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys] 

        for output in outputs:
            output.set_creator(self) 
        self.inputs = inputs
        self.outputs = outputs 
        return outputs if len(outputs) >1 else outputs[0] #리스트의 원소가 하나라면 첫 번쨰 원소를 반환
    
    def forward(self, xs):
        raise NotImplementedError()
    def backward(self, gys):
        raise NotImplementedError()

In [14]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    def backward(self, gy):
        return gy, gy #상류에서 흘러오는 미분값을 그대로 전달

In [15]:
def add(x0, x1):
    return Add()(x0, x1)

x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1) # Add 클래스 생성 과정이 감춰짐

print(y.data)

5


In [16]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx
    
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [17]:
def square(x):
    return Square()(x) 
def exp(x):
    return Exp()(x)

In [18]:
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y))
z.backward()

print(z.data)
print(x.grad)
print(y.grad)

13.0
4.0
6.0
